# New Section

In [59]:
import json
class GenerateLeafletMap:
    def __init__(self,html_file_name):
        self.html_file_name = html_file_name
        self.html_file = open(self.html_file_name, 'w') 
        self.markers = 0
        self.polygons = 0
        self.polylines = 0
        self.layers = 0
        self.type_data = {}
    def close(self):
        self.html_file.close()

    def set_type_data_dict(self,type_data_dict):
        self.type_data = type_data_dict

    def genMapHTMLTop(self):
        self.html_file.write('<!DOCTYPE html>\n')
        self.html_file.write('<html>\n')
        self.html_file.write('<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n')
        self.html_file.write('<title>map title</title>\n')
        self.html_file.write('<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.css" rel="stylesheet" />\n')
        self.html_file.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.js"></script>\n')
        self.html_file.write('<script src="https://code.jquery.com/jquery-2.1.4.min.js"></script>\n')
        self.html_file.write('<style type="text/css">#my-map {\n')
        self.html_file.write('  width:960px;\n')
        self.html_file.write('  height:700px;\n')
        self.html_file.write('}\n')
        self.html_file.write('h1 {\n')
        self.html_file.write('  color: #AA0000;\n')
        self.html_file.write('  font-size: 24px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h2 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 20px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h3 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 16px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')    
        self.html_file.write('p {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('li {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')  
        self.html_file.write('th {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('td {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('</style>\n')
        self.html_file.write('</head>\n')
        self.html_file.write('<body>\n')

    def genMapHTMLH1(self,map_title):
        self.html_file.write('<h1>'+map_title+'</h1>\n')
        self.html_file.write('<div id="description">description</div>\n')
        self.html_file.write('<div id="latitude">latitude</div>\n')
        self.html_file.write('<div id="longitude">longitude</div>\n')
        self.html_file.write('sight: <div id="sight">sight</div>\n')
        self.html_file.write('hearing: <div id="hearing">hearing</div>\n')
        self.html_file.write('smell: <div id="smell">smell</div>\n')
        self.html_file.write('feeling: <div id="feeling">feeling</div>\n')
                    
    def genMapHTMLScriptTop(self): 
        self.html_file.write('  <div id="my-map"></div>\n')
        self.html_file.write('    <script>\n')
        self.html_file.write('    // This file was generated by this R program:  https://github.com/jeffblackadar/oc-transpo-maps/blob/master/oc_4_gen_html.r\n')
        self.html_file.write('var map;\n')
        self.html_file.write("map = L.map('my-map');\n")
        self.html_file.write('var markers = [];\n')
        self.html_file.write('var polygons = [];\n')
        self.html_file.write('var markers_data = [];\n')
        self.html_file.write('var polygons_data = [];\n')

        self.html_file.write("// Thank you:\n")
        self.html_file.write("// https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula\n")
        self.html_file.write("function distance(lat1, lon1, lat2, lon2) {\n")
        self.html_file.write("  var p = 0.017453292519943295;    // Math.PI / 180\n")  
        self.html_file.write("  var c = Math.cos;\n")
        self.html_file.write("  var a = 0.5 - c((lat2 - lat1) * p)/2 + \n")
        self.html_file.write("          c(lat1 * p) * c(lat2 * p) * \n")
        self.html_file.write("          (1 - c((lon2 - lon1) * p))/2;\n")
        self.html_file.write("  return 12742 * Math.asin(Math.sqrt(a)); // 2 * R; R = 6371 km\n")
        self.html_file.write("}\n")

        self.html_file.write("//https://stackoverflow.com/questions/31790344/determine-if-a-point-reside-inside-a-leaflet-polygon\n")
        self.html_file.write("function isMarkerInsidePolygon(marker, poly) {\n")
        self.html_file.write("    var polyPoints = poly.getLatLngs();       \n")
        self.html_file.write("    var x = marker.getLatLng().lat, y = marker.getLatLng().lng;\n")

        self.html_file.write("    var inside = false;\n")
        self.html_file.write("    for (var i = 0, j = polyPoints.length - 1; i < polyPoints.length; j = i++) {\n")
        self.html_file.write("        var xi = polyPoints[i].lat, yi = polyPoints[i].lng;\n")
        self.html_file.write("        var xj = polyPoints[j].lat, yj = polyPoints[j].lng;\n")

        self.html_file.write("        var intersect = ((yi > y) != (yj > y)) && (x < (xj - xi) * (y - yi) / (yj - yi) + xi);\n")
        self.html_file.write("        if (intersect) inside = !inside;\n")
        self.html_file.write("    }\n")

        self.html_file.write("    return inside;\n")
        self.html_file.write("};\n")

        self.html_file.write("function update_per_proximity(marker) {\n")
        self.html_file.write("    var marker_lat = marker.getLatLng().lat, marker_lon = marker.getLatLng().lng;\n")


        self.html_file.write("// Polygons - check if marker is inside.  If so, light it up.\n")  
        self.html_file.write("    for (cn=0;cn<polygons.length;cn++){\n")
        self.html_file.write("        if(isMarkerInsidePolygon(marker, polygons[cn])==true){\n")
        self.html_file.write("            polygons[cn].setStyle({fillOpacity: 0.7, weight:2,opacity: 0.3});\n")
        self.html_file.write("            document.getElementById('sight').innerHTML = polygons_data[cn].sight;\n")
        self.html_file.write("            document.getElementById('hearing').innerHTML = polygons_data[cn].hearing;\n")
        self.html_file.write("            document.getElementById('feeling').innerHTML = polygons_data[cn].feeling;\n")
        self.html_file.write("            document.getElementById('smell').innerHTML = polygons_data[cn].smell;\n")                                                                  
        self.html_file.write("        }\n")
        self.html_file.write("    }\n")

        self.html_file.write("    for (cn=0;cn<markers.length;cn++){\n")
        self.html_file.write("        dist_to = distance(marker_lat, marker_lon,markers[cn].getLatLng().lat,markers[cn].getLatLng().lng)\n")
        self.html_file.write("        // adjust distance for proximity sensitivity\n")
        self.html_file.write("        if(dist_to < 0.1){\n")
        self.html_file.write("            if(cn<13){\n")
        self.html_file.write("                markers[cn].setIcon(iconRCH);\n")
        self.html_file.write("                markers[cn]._popup.setContent('RCH  <audio controls><source src=\"sounds/rch.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(cn==13){\n")
        self.html_file.write("                markers[cn].setIcon(iconFurnace);\n")
        self.html_file.write("                markers[cn]._popup.setContent('Furnace  <audio controls><source src=\"sounds/furnace.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(cn==14){\n")
        self.html_file.write("                markers[cn].setIcon(iconBusiness);\n")
        self.html_file.write("                markers[cn]._popup.setContent('Business  <audio controls><source src=\"sounds/business.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(cn>14){\n")
        self.html_file.write("                markers[cn].setIcon(iconHouse);\n")
        self.html_file.write("                markers[cn]._popup.setContent('House  <audio controls><source src=\"sounds/house.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")    
        self.html_file.write("        document.getElementById('sight').innerHTML = markers_data[cn].sight;\n")
        self.html_file.write("        document.getElementById('hearing').innerHTML = markers_data[cn].hearing;\n")
        self.html_file.write("        document.getElementById('feeling').innerHTML = markers_data[cn].feeling;\n")
        self.html_file.write("        document.getElementById('smell').innerHTML = markers_data[cn].smell;\n")                                                                  
        self.html_file.write("        } // if dist_to\n")
        self.html_file.write("    } // for\n")

        self.html_file.write("} // function\n")                      
      
        self.html_file.write('var iconHouse = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/house.png",\n')
        self.html_file.write('    //shadowUrl: "leaf-shadow.png",\n')

        self.html_file.write('    iconSize:     [30, 37], // size of the icon\n')
        self.html_file.write('    //shadowSize:   [50, 64], // size of the shadow\n')
        self.html_file.write('    iconAnchor:   [15, 36], // point of the icon which will correspond to markers location\n')
        self.html_file.write('    //shadowAnchor: [4, 62],  // the same for the shadow\n')
        self.html_file.write('    popupAnchor:  [-3, -38] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('var iconRCH = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/rch.png",\n')
        self.html_file.write('    iconSize:     [35, 33], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 32], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -34] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')
      
        self.html_file.write('var iconFurnace = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/furnace.png",\n')
        self.html_file.write('    iconSize:     [34, 38], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 37], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -39] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconBusiness = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/business.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconTravel = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/travel.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconComputer = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/computer.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconQuestion = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/question.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')


                    
    def genMapHTMLMapTop(self):
        self.html_file.write("\n")
                      
        self.html_file.write("      //basemap.fitBounds(geojson.getBounds());\n")
        self.html_file.write("      map.setView([45.468, -75.553], 15);\n")
        self.html_file.write("      basemap.addTo(map);\n")
        self.html_file.write("\n")
        self.html_file.write("      // add legend control layers - global variable with (null, null) allows indiv basemaps and overlays to be added inside functions below\n")
        self.html_file.write("      var controlLayers = L.control.layers(null, null, {\n")
        self.html_file.write('        position: "topleft",\n')
        self.html_file.write("        collapsed: false // false = open by default\n")
        self.html_file.write("      }).addTo(map);\n")
        self.html_file.write("\n")
        #writeLines("      var markers = [];", self$outputFileHtmlCon)
                      

    def genHTMLBaseMap(self):
        self.html_file.write('  window.onload = function() {\n')
        self.html_file.write("    var basemap = L.tileLayer('https://{s}.tile.osm.org/{z}/{x}/{y}.png', {\n")
        self.html_file.write("      attribution: '&copy; "+'<a href="https://osm.org/copyright">OpenStreetMap</a>'+"'\n")
        self.html_file.write("    });\n")

    def genMarker(self,marker_lat, marker_long, marker_type):
        type_data_row = self.type_data[marker_type.lower()]
        type_data_row_json = json.loads(type_data_row)  
        self.html_file.write('  markers.push(L.marker(['+str(marker_lat)+','+str(marker_long)+'], {icon: iconQuestion}).addTo(map));\n')
        self.html_file.write('     markers['+str(self.markers)+"].bindPopup('"+str(marker_type)+"').openPopup();\n")
        self.html_file.write('     markers_data.push('+type_data_row+')\n')
        self.markers = self.markers +1


    def genPolygon(self,polygon_coods_json, polygon_type):
        # Thanks to: https://gis.stackexchange.com/questions/290609/entering-manual-coordinates-to-draw-polygon-in-leaflet
        fill_color="black"
        type_data_row = self.type_data[polygon_type.lower()]
        type_data_row_json = json.loads(type_data_row)

        fill_color = type_data_row_json['color']
        #if(polygon_type.lower()=="hill"):
        #    fill_color="DarkGreen"
        #if(tolower(polygon_type)=="forest"){fill_color="Green"}
        #if(tolower(polygon_type)=="field"){fill_color="Wheat"}
        #if(tolower(polygon_type)=="road"){fill_color="DarkGrey"}
        #if(tolower(polygon_type)=="creek"){fill_color="Blue"}
        
        self.html_file.write('var coords = '+polygon_coods_json+';\n')
                      #writeLines(paste0('var a = JSON.parse(coords);\n')
        self.html_file.write('polygons.push(L.polygon(coords, {\n')
        self.html_file.write('  color: "white",\n')
        self.html_file.write('  fillColor: "'+fill_color+'",\n')
        self.html_file.write('  weight: 0,\n')
        self.html_file.write('  opacity: 0,\n')
        self.html_file.write('  fillOpacity: 0\n')
        self.html_file.write('}).addTo(map));\n')
        self.html_file.write('     polygons['+str(self.polygons)+"].bindPopup('"+polygon_type+"').openPopup();\n")
        self.html_file.write('     polygons_data.push('+type_data_row+')\n')
        self.html_file.write('\n')
        self.polygons = self.polygons +1


    def genPolyline(p1_lat, p1_long, p2_lat, p2_long):
        self.polylines = self.polylines +1
        self.html_file.write('  var pointA = new L.LatLng(',paste0(p1_lat),',',paste0(p1_long),' );\n')
        self.html_file.write('  var pointB = new L.LatLng(',paste0(p2_lat),',',paste0(p2_long),');\n')
        self.html_file.write('  var pointList = [pointA, pointB];\n')
        self.html_file.write('  \n')
        self.html_file.write('  var polyline'+str(self.polylines)+' = new L.Polyline(pointList, {\n')
        self.html_file.write("      color: 'red',\n")
        self.html_file.write('      weight: 3,\n')
        self.html_file.write('      opacity: 0.5,\n')
        self.html_file.write('      smoothFactor: 1\n')
        self.html_file.write('  });\n')
        self.html_file.write('  polyline'+str(self.polylines)+'.addTo(map);\n')

                    

    def genHTMLEndMap(self):
        self.html_file.write("      controlLayers.addOverlay(basemap, 'OpenStreetMap');\n")
        self.html_file.write("      \n")
        self.html_file.write("   // });\n")

        self.html_file.write("// To simulate walking around I made a draggable marker. This will greatly reduce testing time.\n")
        self.html_file.write("// Thanks to\n")
        self.html_file.write("https://stackoverflow.com/questions/57384822/leaflet-detect-when-marker-goes-into-and-out-of-a-circle\n")
        self.html_file.write("var test_marker = new L.marker([45.4672641365588, -75.57309653282166],{\n")
 
        self.html_file.write("    draggable: true,\n")
        self.html_file.write("    autoPan: true\n")
        self.html_file.write("}).addTo(map);\n")
        self.html_file.write("test_marker.setIcon(iconComputer)\n")

        self.html_file.write("test_marker.on('dragend', function (e) {\n")
        self.html_file.write("    document.getElementById('latitude').innerHTML = test_marker.getLatLng().lat;\n")
        self.html_file.write("    document.getElementById('longitude').innerHTML = test_marker.getLatLng().lng;\n")
        self.html_file.write("    document.getElementById('description').innerHTML = distance(test_marker.getLatLng().lat,test_marker.getLatLng().lng,markers[13].getLatLng().lat,markers[13].getLatLng().lng);\n")

        self.html_file.write("    update_per_proximity(test_marker)\n")


        self.html_file.write("});\n")

        self.html_file.write("  };\n")


        self.html_file.write("   // https://gis.stackexchange.com/questions/182068/getting-current-user-location-automatically-every-x-seconds-to-put-on-leaflet\n")
        self.html_file.write("var current_position, current_accuracy;\n")
        self.html_file.write("function onLocationFound(e) {\n")
        self.html_file.write("    // if position defined, then remove the existing position marker and accuracy circle from the map\n")
        self.html_file.write("    if (current_position) {\n")
        self.html_file.write("        map.removeLayer(current_position);\n")
        #writeLines(paste0("       map.removeLayer(current_accuracy);\n")
        self.html_file.write("    }\n")
        self.html_file.write("    var radius = e.accuracy / 2;\n")
        self.html_file.write("    current_position = L.marker(e.latlng).addTo(map)\n")
        self.html_file.write('        .bindPopup("You are within " + radius + " meters from this point");\n')
        self.html_file.write("    current_position.setIcon(iconTravel)\n")
        #writeLines(paste0("current_accuracy = L.circle(e.latlng, radius).addTo(map);\n")
        self.html_file.write("    update_per_proximity(current_position);\n")
        self.html_file.write("}\n")
        self.html_file.write("function onLocationError(e) {\n")
        self.html_file.write("    alert(e.message);\n")
        self.html_file.write("}\n")
        self.html_file.write("map.on('locationfound', onLocationFound);\n")
        self.html_file.write("map.on('locationerror', onLocationError);\n")
        self.html_file.write("// wrap map.locate in a function    \n")
        self.html_file.write("function locate() {\n")
        self.html_file.write("    map.locate({setView: true, maxZoom: 16});\n")
        self.html_file.write("}\n")

        self.html_file.write("// call locate every 3 seconds... forever\n")
        self.html_file.write("setInterval(locate, 3000000);\n")

        self.html_file.write("</script>\n")
                    
    def genHTMLEndBody(self):
        self.html_file.write("<hr>\n")
        #self.html_file.write("<p>Page last modifed: ", format(Sys.Date(),"%d %B, %Y"),"</p>\n")
        self.html_file.write("</body>\n")
        self.html_file.write("</html>\n")

    def genLayer(self,geojsonfile):
        self.layers = self.layers +1
        #print(paste0("To map for ", railYear, ", adding: "))
        #print(output_dbRows_route_style[i_route_style, 1])
        layerName = "layer" + str(self.layers)
                      
        self.html_file.write("\n")
        self.html_file.write("// layer for "+geojsonfile+"\n")
        self.html_file.write("var ",layerName,";\n")
                      
        self.html_file.write("      $.ajax({\n")
        self.html_file.write("        type: 'POST',\n")
        self.html_file.write("        url: '",geojsonfile,"',\n")

        self.html_file.write("        dataType: 'json',\n")
        self.html_file.write("        success: function(response) {\n")
        self.html_file.write("          ",layerName," = L.geoJson(response, {\n")
                      
        self.html_file.write("             onEachFeature: function (feature, layer) {\n")

        self.html_file.write("                 layer.bindPopup('<p>'+feature.properties.RTE_TYPE+'</p><p>'+feature.properties.MODE+'</p>');\n")  
                      
        self.html_file.write("               }\n")
        self.html_file.write("             });\n")
        self.html_file.write("          ",layerName,".setStyle({\n")
        self.html_file.write("            color: 'purple',\n")
        self.html_file.write("            weight: 2,\n")
        self.html_file.write('            opacity: 1\n')
        self.html_file.write("          });\n")
        self.html_file.write("          ",layerName,".addTo(map);\n")
        self.html_file.write("          controlLayers.addOverlay(",layerName,", '<span style=",'"color: purple"',">route</span>');\n")

        self.html_file.write("        }\n")
        self.html_file.write("      });\n")
        self.html_file.write("\n")
                      

gm = GenerateLeafletMap("userloc.html")
gm.genMapHTMLTop()


type_data_dict={
    "hill":'{"color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
    "rch":'{"color":"","sight":"a flat circle","hearing":"wind blowing through trees","smell":"ash","feeling":"cool, damp"}',
    "forest":'{"color":"green","sight":"level terrain, trees","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
    "field":'{"color":"wheat","sight":"level terrain, grass","hearing":"wind blowing through grass","smell":"grass","feeling":"warm, sun"}',
    "road":'{"color":"gray","sight":"a muddy road","hearing":"squash of footsteps","smell":"mud","feeling":"sun"}',
    "business":'{"color":"","sight":"a building","hearing":"horses","smell":"smoke","feeling":"cool, damp"}',
    "furnace":'{"color":"","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
    "house":'{"color":"","sight":"a house","hearing":"people talking","smell":"wood smoke","feeling":"cool, damp"}'
}
gm.set_type_data_dict(type_data_dict)

gm.genMapHTMLH1("Hopewell, Pennsylvania")
gm.genMapHTMLScriptTop()
gm.genHTMLBaseMap()
gm.genMapHTMLMapTop()

#gm.genMarker(45.3853076228202, -75.7027063624573, '<a href="http://ofnc.ca/programs/fletcher-wildlife-garden">Fletcher Wildlife Garden</a> pollinator garden.')
#gm.genMarker(45.2177207,-76.19668383, '<h1>Location: Naismith Public School Pollinator Garden.<h1><h2>Sponsor: Almonte Horticultural Society</h2>')
#gm.genMarker(45.6769162,-76.7838167, '<h1>Location: Ross Museum in Forester’s Falls</h1><h2>Sponsor: Beachburg Horticultural Society</h2> <a href="https://www.google.com/maps/dir///@45.6769162,-76.7838167,17z">Directions</a>. Beachburg Horticultural Society partnered with the Ross Museum in Forester’s Falls. Plants are in the Heritage Creek-side garden along with several varieties of shrubs and flowers that attract bees and butterflies. Two bee nests have been installed on a nearby building to provide homes for pollen bees. The Allium bulbs of different sizes and colours we planted will attract visitors and pollen bees. This location is often used as a picnic site and ideal for family photos beside a running stream.')
#gm.genMarker(45.1436508,-76.1471922, '<h1>a href="http://www.cpbheritagemuseum.com/">Location: Victoria School Museum gardens</a></h1><h2>Sponsor: Carleton Place Horticultural Society</h2>')
#gm.genMarker(45.3853076228202,-75.7027063624573,'<h3><a href="http://ofnc.ca/programs/fletcher-wildlife-garden">Fletcher Wildlife Garden</a></h3><p></p><a href="https://www.google.com/maps/place/Fletcher+Wildlife+Garden/@45.3870864,-75.7053924,17z/data=!3m1!4b1!4m5!3m4!1s0x4cce05d7e1130193:0x20a2dc5c1a64075f!8m2!3d45.3870864!4d-75.7032037">Directions from Google Maps</a>')

#gm.genMarker(45.2177207,-76.19668383,'<h3>Naismith Public School Pollinator Garden</h3><p>Sponsor: Almonte Horticultural Society</p><p>260 King St, Almonte, ON K0A 1A0<a href="https://www.google.com/maps/place/Naismith+Memorial+Public+School/@45.2177207,-76.1966838,17z/data=!3m1!4b1!4m5!3m4!1s0x4cd21a82b9e49ee7:0x63ecf7f1f28b0063!8m2!3d45.2177207!4d-76.1944951"><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.6769162,-76.7838167,'<h3>Ross Museum</h3><p>Beachburg Horticultural Society partnered with the Ross Museum in Forester’s Falls. Plants are in the Heritage Creek-side garden along with several varieties of shrubs and flowers that attract bees and butterflies. Two bee nests have been installed on a nearby building to provide homes for pollen bees. The Allium bulbs of different sizes and colours we planted will attract visitors and pollen bees. This location is often used as a picnic site and ideal for family photos beside a running stream.</p><p>2022 Foresters Falls Rd, Foresters Falls, ON K0J 1V0<a href="https://www.google.com/maps/dir///@45.6769162,-76.7838167,17z"><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.1436508,-76.1471922,'<h3><a href="http://www.cpbheritagemuseum.com/">Victoria School Museum</a></h3><p></p><p>267 Edmund St, Carleton Place, ON K7C 3E8<a href=""><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.3853076228202,-75.7027063624573,'<h3><a href="http://ofnc.ca/programs/fletcher-wildlife-garden">Fletcher Wildlife Garden</a></h3><p></p><p>near 865, Prince of Wales Dr, Ottawa, ON K2C 3K1<a href="https://www.google.com/maps/place/Fletcher+Wildlife+Garden/@45.3870864,-75.7053924,17z/data=!3m1!4b1!4m5!3m4!1s0x4cce05d7e1130193:0x20a2dc5c1a64075f!8m2!3d45.3870864!4d-75.7032037"><br>Directions from Google Maps</a><p>')
#gm.genMarker(45.5364896,-77.0975428,'<h3>The Eganville and Area Horticultural Society&apos;s Pollinator Garden</h3><p>The Eganville and Area Horticultural Society&apos;s Pollinator Garden is located on a plot of land generously donated by the township, located adjacent to the Municipal Water Treatment Center at 401 John Street, Eganville. It is clearly marked by a tall Garden Ontario OHA Pollinator Haven sign. Stepping stones inside the garden will help you go through and enjoy the many native species including, chokecherry, bark shrubs, serviceberry trees, cosmos, coneflowers, beebalms, sedums, thyme and asters. The thriving plant life is anchored by 3 pear trees. Pollinators also benefit from aged logs with drilled holes that provide natural bee habitats. Planting began in 2015, and the maturing garden is now abundant, and continues to buzz with activity throughout growing seasons.</p><p>85 John St, Eganville, ON K0J 1T0<a href="https://www.google.com/maps/place/John+Egan+Geo-heritage+Walking+Trail/@45.5364896,-77.0975428,17z/data=!4m12!1m6!3m5!1s0x4cd15eb8d8166041:0x73d40ecc420818ae!2sJohn+Egan+Geo-heritage+Walking+Trail!8m2!3d45.5364671!4d-77.0954829!3m4!1s0x4cd15eb8d8166041:0x73d40ecc420818ae!8m2!3d45.5364671!4d-77.0954829"><br>Directions from Google Maps</a><p>')
#gm.genPolyline(45.4376105533903, -75.6575202941895,45.8376105533903, -75.8575202941895)

gm.genMarker(45.47737824334609, -75.56177057947743,'RCH')
gm.genMarker(45.476518775971925, -75.55807403150058,'RCH')
gm.genMarker(45.47755761992494, -75.55628185896252,'RCH')
gm.genMarker(45.4782746075903, -75.55256888504435,'RCH')
gm.genMarker(45.47742592519542, -75.55107903489827,'RCH')
gm.genMarker(45.47785681425306, -75.549202940825,'RCH')
gm.genMarker(45.47825355011008, -75.54822452135727,'RCH')
gm.genMarker(45.46876341531861, -75.54794596398438,'RCH')
gm.genMarker(45.46880850621683, -75.54622057099333,'RCH')
gm.genMarker(45.468217256597285, -75.54717066794124,'RCH')
gm.genMarker(45.46720550171153, -75.54761720916797,'RCH')
gm.genMarker(45.46775663040807, -75.54870263588802,'RCH')
gm.genMarker(45.46590290154052, -75.55414107338335,'RCH')
gm.genMarker(45.47484907392693, -75.55074863942413,'Furnace')
gm.genMarker(45.474344000000016, -75.544972,'Business')
gm.genMarker(45.47299002135775, -75.54525733469424,'House')
gm.genMarker(45.47211988308255, -75.54545798813426,'House')
gm.genMarker(45.47136101744633, -75.54503437930204,'House')
gm.genMarker(45.47131627755208, -75.54376006396652,'House')
gm.genMarker(45.47065452130986, -75.54485043985424,'House')
gm.genMarker(45.469758350845645, -75.5434314118572,'House')
gm.genMarker(45.47205387423288, -75.54739104768343,'House')
gm.genMarker(45.47152713113501, -75.54933902568612,'House')
gm.genMarker(45.47095955010231, -75.55025388893732,'House')
gm.genPolygon('[[45.46780459902121,-75.5611450837986],[45.4681232420102,-75.55715782840628],[45.469462833076285,-75.5538553364279],[45.47004949919434,-75.55298274674901],[45.4721218558572,-75.5468624876541],[45.4704621579035,-75.54466582771028],[45.4651335148253,-75.55457719758812],[45.46515838186637,-75.56126903917371],[45.46780459902121,-75.5611450837986]]','hill')
gm.genPolygon('[[45.46820056878839,-75.5611517404839],[45.468173026077515,-75.56027962634214],[45.46874307778686,-75.55750643997715],[45.47103679114703,-75.55326124271328],[45.47234527953415,-75.55058033515387],[45.47363503251533,-75.54673578605572],[45.47424058116333,-75.54619227807709],[45.47440330691751,-75.54527508935335],[45.47423958566836,-75.5452804058906],[45.47406134805594,-75.54608168238934],[45.473578590930515,-75.54662120545925],[45.47217783034303,-75.55035293879784],[45.47116267822945,-75.55259753564633],[45.46942405632627,-75.55569963545896],[45.46872200708393,-75.55704151411376],[45.46829466689712,-75.55849090401499],[45.467990717283016,-75.55997509712374],[45.46796739438805,-75.56108164483088],[45.46820056878839,-75.5611517404839]]','road')
gm.genPolygon('[[45.4681271962405,-75.55911710189253],[45.46863828916885,-75.55692781343221],[45.46914342801491,-75.55607730179683],[45.47090593465267,-75.55276104653034],[45.47405013523158,-75.54538356869108],[45.47117473244954,-75.54483669315671],[45.472252686787364,-75.54650901569404],[45.472299580981684,-75.54772979070731],[45.4701865541728,-75.55301711709805],[45.469412289742735,-75.55410917603488],[45.46812540834582,-75.55729355863443],[45.46804224374386,-75.55892583923745],[45.4681271962405,-75.55911710189253]]','field')
gm.genPolygon('[[45.47638229037084,-75.5461033600068],[45.476362729665425,-75.54573533856137],[45.47573419685116,-75.54569753604244],[45.47571151927351,-75.54513559279866],[45.47506965446742,-75.54511763303367],[45.47429166694213,-75.54512349669642],[45.469895296949055,-75.5442186232942],[45.469858418888215,-75.54447203048011],[45.4737871713734,-75.54515928255182],[45.47445912682897,-75.54535088591129],[45.47556549155368,-75.54539257023653],[45.475562438055405,-75.54605236076098],[45.47638229037084,-75.5461033600068]]','road')
gm.genPolygon('[[45.476333573917195,-75.54561986738292],[45.47616215022454,-75.5425985796993],[45.4763511012236,-75.54076855373471],[45.47250084572841,-75.54071927860043],[45.470741189170134,-75.54419114010034],[45.47583244075082,-75.54501524897108],[45.475869385356894,-75.54561553577513],[45.476333573917195,-75.54561986738292]]','field')
gm.genPolygon('[[45.47901543139081,-75.56395674215784],[45.481970852150845,-75.5489976213172],[45.480300752297275,-75.54870247008273],[45.47711431776411,-75.54748633241826],[45.47676782259542,-75.55312446533088],[45.477350140512065,-75.55539522274616],[45.476551293536936,-75.55837027031522],[45.47769098041394,-75.56136045217362],[45.47901543139081,-75.56395674215784]]','forest')

gm.genHTMLEndMap()
gm.genHTMLEndBody()

gm.close()

# New Section

In [11]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)")+"userloc.html")

https://pw3i1ela1td-496ff2e9c6d22116-8000-colab.googleusercontent.com/userloc.html


In [60]:
!python -m http.server 8000

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [21/Apr/2021 15:48:29] "GET /userloc.html HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:48:30] code 404, message File not found
127.0.0.1 - - [21/Apr/2021 15:48:30] "GET /icons/question.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2021 15:48:30] code 404, message File not found
127.0.0.1 - - [21/Apr/2021 15:48:30] "GET /icons/computer.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2021 15:48:30] code 404, message File not found
127.0.0.1 - - [21/Apr/2021 15:48:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2021 15:48:42] code 404, message File not found
127.0.0.1 - - [21/Apr/2021 15:48:42] "GET /icons/house.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2021 15:48:43] code 404, message File not found
127.0.0.1 - - [21/Apr/2021 15:48:43] "GET /icons/house.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2021 15:48:45] code 404, message File not found
127.0.0.1 - - [21/Apr/2021 15:48:45] "GET /icons/rch.png HTTP/1.1" 404